# Trabajo Integrador Individual
## "De Texto Crudo a Insights: Pipeline Completo de Análisis de NLP"

**Fecha de entrega**: Jueves 25 de septiembre  
**Modalidad**: Individual  
**Formato**: Repositorio GitHub con notebook documentado

---

## Parte 1: Construcción del Corpus


### Intento de Corpus literario

Se han revisado varias fuentes posibles de información para cuentos contemporáneos o de autores latinoamericanos pero encontré varios problemas a la hora de seleccionar el corpus de este formato que cumpla con el criterio minimo de volumen para procesarlo...

#### Project Gutenberg
Usando https://www.gutenberg.org/browse/languages/es

Autor elegido: Blasco Ibáñez, Vicente, 1867-1928

No tenia suficiente volumen digerible para cuentos y las obras parecían dispersas y repetidas

No tuve forma de encontrar en una forma concisa autores argentinos o latinoamericanos que estén en Gutenberg y que dispongan de mucho material usable para el proyecto... Podria buscar a mano pero tardaria mucho

Se intentó recolectar datos con
* Cuentos valencianos
https://www.gutenberg.org/ebooks/66514

* La condenada (cuentos)
https://www.gutenberg.org/ebooks/27736

* La Catedral

Entre otros...

#### Biblioteca Digital Argentina
La página no se encuentra disponible, y las alternativas no son lo suficientemente voluminosas o contemporaneas para el target que se busca

#### Cervantes Virtual
Mayoritariamente autores españoles, se buscaban autores latinoamericanos

#### Ciudad Seva
De algun modo no me convenció del todo, y preferí buscar otro tipo de corpus para entonces

### Corpus periodistico

Se decide entonces recolectar información relacionada al tema *tecnología*, con énfasis en el asunto de **privacidad**.

Fuentes elegidas
- https://derechodelared.com/
- https://www.anred.org/
- https://www.pagina12.com.ar/

Se omiten otros sitios periodisticos como Clarin, La Nación e Infobae ya que no proporcionaban suficiente información relevante al filtro que queremos aplicar

---

Yapa - No usables pero interesantes para tener en RSS posiblemente?
- https://www.elladodelmal.com/
- (Listado con más blogs y sitios periodisticos) https://www.genbeta.com/seguridad/17-expertos-blogs-y-newsletter-a-seguir-si-de-verdad-te-interesan-seguridad-y-privacidad

### Resolucion

#### Import de librerias

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import os
import re
import shutil
from google.colab import files
from urllib.parse import urlparse


#### Obtención de listado de URLs
Las urls a probar de mi corpus están guardadas en un .txt dentro del repositorio

In [20]:
# 003_barthou-gonzalo-nlp-integrador/corpus/corpusList.txt
# https://raw.githubusercontent.com/gimb99/PDH_BARTHOU_GONZALO/refs/heads/develop/003_barthou-gonzalo-nlp-integrador/corpus/corpusList.txt

# URL cruda del archivo en GitHub
github_txt_url = "https://raw.githubusercontent.com/gimb99/PDH_BARTHOU_GONZALO/refs/heads/develop/003_barthou-gonzalo-nlp-integrador/corpus/corpusList.txt"

# Descargar y procesar las líneas
response = requests.get(github_txt_url)
lines = response.text.splitlines()
urls = [line.strip() for line in lines if line.strip()]


#### Guardado de .txt como archivos raw

In [25]:
# Crear carpeta para guardar los archivos
folder = "raw_texts"
os.makedirs(folder, exist_ok=True)

# Función para limpiar el nombre del archivo
def clean_filename(title, index):
    title = re.sub(r'[^\w\s-]', '', title).strip().replace(' ', '_')
    return f"{index:03d}_{title[:50]}.txt"  # Limita el nombre a 50 caracteres

# Recorrer las URLs
for i, url in enumerate(urls, start=1):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Extraer título y texto (ajustar según el sitio)
        domain = urlparse(url).netloc
        try:
          if "derechodelared.com" in domain:
              title = soup.find("h1", class_="block-entry-title").get_text(strip=True)
          elif "clarin.com" in domain:
              title = soup.find("h1", class_="title").get_text(strip=True)
          else:
              title = soup.find("h1").get_text(strip=True)
        except:
          title = f"articulo_{i}"


        paragraphs = soup.find_all("p")
        text = "\n".join([p.get_text(strip=True) for p in paragraphs])

        # Guardar como archivo .txt
        filename = clean_filename(title, i)
        filepath = os.path.join(folder, filename)

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(f"URL: {url}\n")
            f.write(f"Título: {title}\n\n")
            f.write(text)

        print(f"✅ Guardado: {filename}")

    except Exception as e:
        print(f"❌ Error con {url}: {e}")

✅ Guardado: 001_Tiktok_se_enfrenta_a_una_posible_multa_de_27_millo.txt
✅ Guardado: 002_Manzana_podrida_decenas_de_exespías_israelíes_fuer.txt
✅ Guardado: 003_El_auge_del_tecnofeudalismo.txt
✅ Guardado: 004_Qué_hay_detrás_de_las_colas_para_la_venta_de_iris.txt
✅ Guardado: 005_Sam_Altman_creador_de_ChatGPT_y_GPT-4_mi_peor_temo.txt
✅ Guardado: 006_Natalia_Zuazo_el_problema_es_cuando_el_avance_de_l.txt
✅ Guardado: 007_Espionaje_ilegal_del_Ministerio_de_Seguridad_porte.txt
✅ Guardado: 008_Pensamiento_crítico_De_verdad_sabemos_qué_hacemos_.txt
✅ Guardado: 009_CABA_la_Legislatura_porteña_debate_un_polémico_pro.txt
✅ Guardado: 010_WhatsApp_no_cerrará_cuentas_pero_limitará_funcione.txt
✅ Guardado: 011_Polisis_una_IA_que_se_lee_las_políticas_de_privaci.txt
✅ Guardado: 012_La_AEPD_presenta_nuevos_recursos_para_fomentar_la_.txt
✅ Guardado: 013_Chat_Control_el_plan_europeo_para_escanear_tus men.txt
✅ Guardado: 014_La_NSA_compra_datos_de_navegación_de_los_usuarios .txt
✅ Guardado: 015_Microsoft_mant

##### Textos debug / desarrollo - Ignorar

Elminar carpeta para reintentar

In [24]:
# Eliminar carpeta
# folder = "raw_texts"  # Cambiá por el nombre de tu carpeta

# # Eliminar todos los archivos dentro de la carpeta
# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path):
#             os.remove(file_path)
#             print(f"🗑️ Borrado: {filename}")
#     except Exception as e:
#         print(f"❌ Error con {filename}: {e}")

🗑️ Borrado: 018_La_hora_de_los_nuevos_derechos_y_algo_más.txt
🗑️ Borrado: 011_Polisis_una_IA_que_se_lee_las_políticas_de_privaci.txt
🗑️ Borrado: 007_Espionaje_ilegal_del_Ministerio_de_Seguridad_porte.txt
🗑️ Borrado: 023_Apple_multada_Francia_sanciona_con_162_millones_po.txt
🗑️ Borrado: 017_Una_pelea_por_los_datos_de_los_usuarios.txt
🗑️ Borrado: 016_Pandemia_de_control_digital.txt
🗑️ Borrado: 026_La_era_del_capitalismo_digital_magnates_tecnológic.txt
🗑️ Borrado: 003_El_auge_del_tecnofeudalismo.txt
🗑️ Borrado: 024_Cómo_las_grandes_empresas_tech_lograron_meterse_en.txt
🗑️ Borrado: 015_Microsoft_mantendrá_en_la_UE_los_datos_personales_.txt
🗑️ Borrado: 005_Sam_Altman_creador_de_ChatGPT_y_GPT-4_mi_peor_temo.txt
🗑️ Borrado: 022_Catamarca_y_Telecentro_firman_un_acuerdo_histórico.txt
🗑️ Borrado: 008_Pensamiento_crítico_De_verdad_sabemos_qué_hacemos_.txt
🗑️ Borrado: 002_Manzana_podrida_decenas_de_exespías_israelíes_fuer.txt
🗑️ Borrado: 021_Meta_enfrenta_sanciones_millonarias_en_Nigeria_por.txt
🗑

Guardado de zip

In [28]:
# Guardado de .zip para luego extraerlo y subirlo a github
# shutil.make_archive("raw_texts", "zip", "raw_texts")
# files.download("raw_texts.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#### Guardado de metadata

In [9]:
# Armado metadata.csv
# Lista para guardar los metadatos
metadata = []

for url in urls:
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # Ejemplo de extracción (puede variar según el sitio)
        title = soup.find("h1").get_text(strip=True)
        paragraphs = soup.find_all("p")
        text = " ".join([p.get_text(strip=True) for p in paragraphs])

        metadata.append({
            "url": url,
            "title": title,
            "text": text[:500],  # Guardamos solo los primeros 500 caracteres
            "length": len(text),
        })

    except Exception as e:
        print(f"Error con {url}: {e}")

In [10]:
df = pd.DataFrame(metadata)
df.to_csv("metadata.csv", index=False)


---

## Parte 2: Análisis Técnico - Estructura del Notebook


## Datos de commiteo - Ignorar

In [ ]:
## Para commits, guardar ruta como
## 003_barthou-gonzalo-nlp-integrador/notebook/analisis_integrador.ipynb